In [1]:
import os, code, sys, time, argparse

import numpy as np
import sklearn.neighbors as skn
from sklearn.neighbors import kneighbors_graph, radius_neighbors_graph
import tensorflow as tf

import utils
import nn
from utils import REDSHIFTS, PARAMS_SEED, LEARNING_RATE, RS_TAGS

In [2]:
# data specs
num_particles = 32
redshift_steps = [11,19]
num_rs = len(redshift_steps)
num_rs_layers = num_rs - 1

# load data
X = utils.load_zuni_npy_data(redshifts=redshift_steps, norm_coo=True)[...,:-1]
Y = X[1]
X = X[0]
print('X.shape: {}'.format(X.shape))

LD: X_0.5588_.npy
LD: X_0.0000_.npy
X.shape: (1000, 32768, 6)


In [3]:
''' neighbors stuff
'''
# KNN
def alist_to_indexlist(alist):
    """ Reshapes adjacency list for tensorflow gather_nd func
    alist.shape: (B, N, K)
    ret.shape:   (B*N*K, 2)
    """
    batch_size, N, K = alist.shape
    id1 = np.reshape(np.arange(batch_size),[batch_size,1])
    id1 = np.tile(id1,N*K).flatten()
    out = np.stack([id1,alist.flatten()], axis=1).astype(np.int32)
    return out

def knn_alist(x, K=14):
    return kneighbors_graph(x[...,:3], K, include_self=True)

def get_kneighbor_alist(X_in, K=14):
    """ search for K nneighbors, and return offsetted indices in adjacency list
    No periodic boundary conditions used

    Args:
        X_in (numpy ndarray): input data of shape (mb_size, N, 6)
    """
    mb_size, N, D = X_in.shape
    adj_list = np.zeros([mb_size, N, K], dtype=np.int32)
    for i in range(mb_size):
        # this returns indices of the nn
        #graph_idx = kneighbors_graph(X_in[i, :, :3], K, include_self=True).indices
        graph_idx = knn_alist(X_in[i], K).indices
        graph_idx = graph_idx.reshape([N, K]) #+ (N * i)  # offset idx for batches
        adj_list[i] = graph_idx
    return adj_list

def kgraph_conv(h, adj, K):
    """ Graph convolution op for KNN-based adjacency lists
    build graph tensor with gather_nd, and take
    mean on KNN dim: mean((mb_size, N, K, k_in), axis=2)
    Args:
        h: data tensor, (mb_size, N, k_in)
        adj: adjacency index list, for gather_nd (*, 2)
        K (int): num nearest neighbors
    """
    dims = tf.shape(h)
    mb = dims[0]; n  = dims[1]; d  = dims[2];
    rdim = [mb,n,K,d]
    nn_graph = tf.reduce_mean(tf.reshape(tf.gather_nd(h, adj), rdim), axis=2)
    return nn_graph

def knn_fn(h_in): # for tf.py_func
    return alist_to_indexlist(get_kneighbor_alist(h_in, K))

In [4]:
def rad_fn(h_in, rad=0.08):
    return radius_neighbors_graph(h_in[...,:3], rad, include_self=True)

def knn_alist(x, K=14):
    return kneighbors_graph(x[...,:3], K, include_self=True)

'''
csr: indptr, indices, data

for sample i

rad_graph indices at: 
rad_graph_indices = indices[indptr[i]:indptr[i+1]]

'''

'\ncsr: indptr, indices, data\n\nfor sample i\n\nrad_graph indices at: \nrad_graph_indices = indices[indptr[i]:indptr[i+1]]\n\n'

In [5]:
K = 8
R = 0.08
j = 375
x = X[j]

# knn
x_k = knn_alist(np.copy(x), K) # (N * K) == (32768 * 8) == (262144)
k_indptr = x_k.indptr
k_indices = x_k.indices

# rad
x_r = rad_fn(np.copy(x), R)
r_indptr = x_r.indptr
r_indices = x_r.indices

In [89]:
def get_sample_idx(csr, j):
    return csr.indices[csr.indptr[j]:csr.indptr[j+1]]

def get_adj(x, idx):
    return x[np.sort(idx)]

In [57]:
k_idx = get_sample_idx(x_k, 0)
r_idx = get_sample_idx(x_r, 0)

In [6]:
'''
Now, how to do graph conv
'''
N = 32**3
sess = tf.InteractiveSession()
#k_idx = np.reshape(k_indices, (N, K))
#xk_gathered = tf.gather(x, k_idx).eval() # same as x[k_idx]

In [7]:
x_r = x_r.astype(np.float32)

In [8]:
#diffs = np.sum(xr_dense, axis=-1, keepdims=True)[...,None]
diffs = np.diff(x_r.indptr)[...,None]
diffs.shape

(32768, 1)

In [9]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

def tf_sparse_matmul(sparse_mat, x, diffs):
    return tf.sparse_tensor_dense_matmul(sparse_mat, x) / diffs

In [10]:
rad_sparseT = convert_sparse_matrix_to_sparse_tensor(x_r)
spm = tf_sparse_matmul(rad_sparseT, x, diffs)

In [12]:
xr_dense = np.array(x_r.todense()).astype(np.float32)
np_matmul = np.matmul(xr_dense, x) / diffs

In [20]:
tf_matmul = spm.eval()

In [21]:
tf_matmul

array([[ 0.08974745,  0.92554116,  0.02650417,  0.40986484, -1.1076114 ,
         0.06140422],
       [ 0.06966576,  0.9270991 ,  0.97068673,  0.22520797, -1.0257113 ,
         0.0207623 ],
       [ 0.11268491,  0.9101092 ,  0.03641011,  0.4703874 , -1.145686  ,
         0.14578038],
       ...,
       [ 0.9046952 ,  0.89274216,  0.9599931 ,  0.02504629, -0.6929508 ,
         0.02516849],
       [ 0.97368205,  0.87946206,  0.9466478 ,  0.5004067 , -0.78527445,
        -0.12592651],
       [ 0.03181459,  0.872992  ,  0.93856347,  0.20551766, -0.7957768 ,
        -0.08066343]], dtype=float32)

In [23]:
np.allclose(tf_matmul, np_matmul)

False

In [28]:
np.allclose(tf_matmul, np_matmul, atol=1e06)

True